<a href="https://colab.research.google.com/github/laura-katiuscia/2-Projeto/blob/master/Machine_Learning_com_Analise_de_dados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Entendo conceito de Pipeline com Python para Machine Learning (e um conceito que

# pode ser usada nas ling. Java, Python etc..Ela começa antes DATALAKE E DATAHOUSE).

# 📌 Pipeline simples (funções em sequência)
def limpar_dados(dados):
    return [x for x in dados if x is not None]

def normalizar(dados):
    max_val = max(dados)
    return [x / max_val for x in dados]

def treinar_modelo(dados):
    print("Treinando modelo com:", dados)

# Pipeline
dados = [10, 20, None, 30]
dados = limpar_dados(dados)
dados = normalizar(dados)
treinar_modelo(dados)

#📌 Ideia-chave: saída de uma função vira entrada da próxima.



In [ ]:
# Carregar arquivo colab
from google.colab import files
uploaded = files.upload()


In [ ]:
# lê o arquivo já carregado ✔️ Muito usado quando você quer limitar leituras de arquivos grandes.
from itertools import islice

with open('producao_alimentos.csv', 'r') as f:
    for linha in islice(f, 6):
        print(linha, end='')



In [ ]:
#Essa parte toda mas segunda parte do código serve para testar a conectividade
import csv
import sqlite3

#Cria novo banco de dados
conn = sqlite3.connect('dsadb.db')

# Adiciona esta linha para dropar a tabela se ela já existir
conn.execute('DROP TABLE IF EXISTS producao')

#Cria tabela para armazenar dos dados do arquivo producao_alimentos.csv
conn.execute('''CREATE TABLE producao (
            produto TEXT,
            quantidade INTEGER,
            preco_media REAL,
            receita_media REAL
)''')
conn.commit()
conn.close()

In [ ]:
#abre o arquivo CSV com os dados de produção de alimentos
with open('producao_alimentos.csv', 'r') as file:

#Cria um leitor de CSV para ler o arquivo
  reader = csv.reader(file)

  # pula primeira linha que contém os cabeçalhos das colunas
  next (reader)

  #Conecta ao banco de dados
  conn = sqlite3.connect('dsadb.db')


  for row in reader:

      conn.execute('INSERT INTO producao (produto, quantidade, preco_media, receita_media) VALUES (?, ?, ?, ?)', row)


  conn.commit()
  conn.close()

  print("Job Concluído com Sucesso!")

In [ ]:
#Esse código foi feito para responder o que foi pedido carrega somente quantidade
#prouzida maior que 10 da coluna quantidade_produzida_kls  OBS: Essa atividade
#JOB_V3.PY QUE E TRANSFORMAÇÃO DOS DADOS.
import csv
import sqlite3

# Função para remover o ponto nos dados da última coluna
def remove_ponto(valor):
    return int(valor.replace('.', '')) #Essa função usa replace que indetifica
    #ponto e substitui por virgula e no final converte para número inteiro.

# Abre o arquivo CSV com os dados da produção de alimentos
with open('producao_alimentos.csv', 'r') as file:

    # Cria um leitor de CSV para ler o arquivo
    reader = csv.reader(file)

    # Pula a primeira linha, que contém os cabeçalhos das colunas
    next(reader)

    # Conecta ao banco de dados
    conn = sqlite3.connect('dsadb.db')

    # Deleta a tabela existente, se houver
    conn.execute('DROP TABLE IF EXISTS producao')  #Aula Prática 04, TRANSFORMAÇÃO
    # DE DADOS, tirou os pontos que não aparecia a casa decimal dos zeros dos valores.

    # Cria uma nova tabela para armazenar os dados de produção de alimentos
    conn.execute('''CREATE TABLE producao (
                    produto TEXT,
                    quantidade INTEGER,
                    preco_medio REAL,
                    receita_total INTEGER,
                    margem_lucro REAL
                )''')

    #AQUI NO EXECUTE E DIFERENÇA DO EXERCICIO 03 PARA 04 ONDE SÓ ACRESCENTA coluna
    # com nome margem_lucro com resultado das colunas existentes que fez calculo

    # Insere cada linha do arquivo com quantidade maior que 10 na tabela do banco de dados
    #Aqui mantên a regra de negócio
    for row in reader:
        if int(row[1]) > 10:

            # Remove o ponto do valor da última coluna e converte para inteiro
            row[3] = remove_ponto(row[3]) #ùltima linha que e receita_total e
            #chama função criada com nome remove_ponto

            margem_lucro = (row[3] / float(row[1])) - float (row[2]) # Calculo 1º subtrai linha 1 menos a linha 2
            #row.append(margem_lucro)

            # Insere o registro no banco de dados sendo que margem de lucro não existe na origem isso deixa ele pronto pra uso
            #coluna margem e pra colocar resultado das colunas já existentes
            conn.execute('INSERT INTO producao (produto, quantidade, preco_medio, receita_total, margem_lucro) VALUES (?, ?, ?, ?, ?)', (row[0], row[1], row[2], row[3], margem_lucro))

    conn.commit()
    conn.close()

print("Job Concluído com Sucesso! Para vê o resultado abra no ")

In [ ]:
# 📌 Pipeline com Pandas (Análise de Dados), muito usado no pré-processamento de dados %%

import pandas as pd

df = pd.DataFrame({
    "idade": [20, 25, None, 30],
    "salario": [2000, None, 3000, 4000]
})

pipeline = (
    df
    .dropna()
    .assign(
        idade_norm=lambda x: x["idade"] / x["idade"].max(),
        salario_norm=lambda x: x["salario"] / x["salario"].max()
    )
)

print(pipeline)

#📌 Ideal para ETL (Extract, Transform, Load).


In [ ]:
# 🚀 Pipeline profissional com scikit-learn (Machine Learning)

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris

# Dados
X, y = load_iris(return_X_y=True)

X_treino, X_teste, y_treino, y_teste = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Pipeline
pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("modelo", LogisticRegression(max_iter=200))
])

# Treinar
pipeline.fit(X_treino, y_treino)

# Avaliar
print("Acurácia:", pipeline.score(X_teste, y_teste))

#🔥 O que acontece aqui? StandardScaler → normaliza os dados, LogisticRegression → treina o modelo, Tudo acontece automaticamente em ordem

# 📌 Evita erros, vazamento de dados e facilita deploy.


In [ ]:
# 4️⃣ Pipeline com colunas diferentes (numéricas e categóricas), Muito comum em projetos reais.

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier

preprocessador = ColumnTransformer([
    ("num", StandardScaler(), ["idade", "salario"]),
    ("cat", OneHotEncoder(), ["sexo"])
])

pipeline = Pipeline([
    ("prep", preprocessador),
    ("modelo", RandomForestClassifier())
])

#Esse pipeline NÃO gera resposta sozinho. A resposta do modelo só aparece depois que você treina e faz uma predição.

# 👉 A resposta final do pipeline será: Uma classe prevista, por exemplo: 0 → não compra, 1 → compra


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Criando um dataset dummy para treino (com idade, salario, sexo e um target)
# Isso é necessário porque o pipeline com ColumnTransformer espera dados com essas colunas para ser treinado
data_treino = pd.DataFrame({
    "idade": [25, 35, 45, 22, 38, 50],
    "salario": [2500, 3500, 4500, 2200, 3800, 5000],
    "sexo": ["F", "M", "F", "M", "F", "M"],
    "target": [0, 1, 0, 1, 0, 1]
})

# Separando features (X) e target (y) para o treino
X_treino_dummy = data_treino[["idade", "salario", "sexo"]]
y_treino_dummy = data_treino["target"]

# Treinar o pipeline com os dados dummy
pipeline.fit(X_treino_dummy, y_treino_dummy)

novo_cliente = pd.DataFrame({
    "idade": [30],
    "salario": [4000],
    "sexo": ["F"]
})

resposta = pipeline.predict(novo_cliente)
probabilidade = pipeline.predict_proba(novo_cliente)

print("Classe prevista:", resposta)
print("Probabilidade:", probabilidade)

#Resposta classe prevista 0 sig. que ele não vai comprar